In [7]:
"""
REALITY CHECK: Is RDW/MCHC truly superior or just statistical noise?
This script performs rigorous testing to determine genuine clinical value.
"""

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def reality_check_biomarker(df, outcome='mort_30d'):
    """
    Comprehensive reality check for RDW/MCHC superiority claims
    """
    print("="*80)
    print("REALITY CHECK: IS RDW/MCHC ACTUALLY BETTER THAN NLR?")
    print("="*80)
    
    results = {}
    
    # =========================================================================
    # TEST 1: BASIC PERFORMANCE WITH CONFIDENCE INTERVALS
    # =========================================================================
    print("\n1. BASIC PERFORMANCE COMPARISON")
    print("-"*40)
    
    # Calculate AUCs with bootstrap CIs
    def bootstrap_auc(y_true, y_scores, n_bootstraps=1000):
        aucs = []
        n_samples = len(y_true)
        
        for _ in range(n_bootstraps):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            if len(np.unique(y_true[indices])) < 2:
                continue
            auc = roc_auc_score(y_true[indices], y_scores[indices])
            aucs.append(auc)
        
        return np.mean(aucs), np.percentile(aucs, [2.5, 97.5])
    
    # Complete case analysis
    complete_df = df[['rdw_to_mchc', 'nlr', outcome]].dropna()
    
    rdw_mchc_auc, rdw_mchc_ci = bootstrap_auc(
        complete_df[outcome].values,
        complete_df['rdw_to_mchc'].values
    )
    
    nlr_auc, nlr_ci = bootstrap_auc(
        complete_df[outcome].values,
        complete_df['nlr'].values
    )
    
    print(f"RDW/MCHC: {rdw_mchc_auc:.3f} (95% CI: {rdw_mchc_ci[0]:.3f}-{rdw_mchc_ci[1]:.3f})")
    print(f"NLR:      {nlr_auc:.3f} (95% CI: {nlr_ci[0]:.3f}-{nlr_ci[1]:.3f})")
    print(f"Difference: {rdw_mchc_auc - nlr_auc:.3f}")
    
    # Check if CIs overlap
    if rdw_mchc_ci[0] > nlr_ci[1]:
        print("✓ Statistically significant superiority (non-overlapping CIs)")
        results['basic_significant'] = True
    else:
        print("✗ Confidence intervals overlap - difference may not be significant")
        results['basic_significant'] = False
    
    # =========================================================================
    # TEST 2: CLINICAL SIGNIFICANCE
    # =========================================================================
    print("\n2. CLINICAL SIGNIFICANCE CHECK")
    print("-"*40)
    
    # What's the minimum clinically important difference?
    min_clinical_diff = 0.05  # 5% AUC improvement typically considered meaningful
    
    actual_diff = rdw_mchc_auc - nlr_auc
    
    if actual_diff >= min_clinical_diff:
        print(f"✓ Clinically significant ({actual_diff:.3f} ≥ {min_clinical_diff})")
        results['clinically_significant'] = True
    else:
        print(f"✗ Not clinically significant ({actual_diff:.3f} < {min_clinical_diff})")
        results['clinically_significant'] = False
    
    # =========================================================================
    # TEST 3: RECLASSIFICATION ANALYSIS
    # =========================================================================
    print("\n3. RECLASSIFICATION IMPACT")
    print("-"*40)
    
    # Fit models
    lr_rdw = LogisticRegression()
    lr_nlr = LogisticRegression()
    
    lr_rdw.fit(complete_df[['rdw_to_mchc']], complete_df[outcome])
    lr_nlr.fit(complete_df[['nlr']], complete_df[outcome])
    
    pred_rdw = lr_rdw.predict_proba(complete_df[['rdw_to_mchc']])[:, 1]
    pred_nlr = lr_nlr.predict_proba(complete_df[['nlr']])[:, 1]
    
    # Risk categories
    def categorize_risk(probs):
        cats = np.zeros(len(probs))
        cats[probs >= 0.1] = 1  # Intermediate risk
        cats[probs >= 0.2] = 2  # High risk
        return cats
    
    cat_rdw = categorize_risk(pred_rdw)
    cat_nlr = categorize_risk(pred_nlr)
    
    # How many patients are reclassified?
    reclassified = cat_rdw != cat_nlr
    n_reclassified = reclassified.sum()
    pct_reclassified = n_reclassified / len(complete_df) * 100
    
    # Are reclassifications correct?
    deaths = complete_df[outcome] == 1
    
    # Correct reclassifications
    correct_up = ((cat_rdw > cat_nlr) & deaths).sum()
    correct_down = ((cat_rdw < cat_nlr) & ~deaths).sum()
    
    # Incorrect reclassifications
    incorrect_up = ((cat_rdw > cat_nlr) & ~deaths).sum()
    incorrect_down = ((cat_rdw < cat_nlr) & deaths).sum()
    
    net_correct = (correct_up + correct_down) - (incorrect_up + incorrect_down)
    
    print(f"Patients reclassified: {n_reclassified} ({pct_reclassified:.1f}%)")
    print(f"Correct reclassifications: {correct_up + correct_down}")
    print(f"Incorrect reclassifications: {incorrect_up + incorrect_down}")
    print(f"Net benefit: {net_correct}")
    
    if net_correct > 0:
        print("✓ Net positive reclassification")
        results['reclassification_positive'] = True
    else:
        print("✗ No net benefit from reclassification")
        results['reclassification_positive'] = False
    
    # =========================================================================
    # TEST 4: NUMBER NEEDED TO TEST
    # =========================================================================
    print("\n4. PRACTICAL CLINICAL IMPACT")
    print("-"*40)
    
    # At optimal cutpoints, how many need testing to identify one death?
    from sklearn.metrics import confusion_matrix
    
    # Find optimal cutpoints
    fpr_rdw, tpr_rdw, thresh_rdw = roc_curve(complete_df[outcome], pred_rdw)
    youden_rdw = tpr_rdw - fpr_rdw
    optimal_rdw = thresh_rdw[np.argmax(youden_rdw)]
    
    fpr_nlr, tpr_nlr, thresh_nlr = roc_curve(complete_df[outcome], pred_nlr)
    youden_nlr = tpr_nlr - fpr_nlr
    optimal_nlr = thresh_nlr[np.argmax(youden_nlr)]
    
    # Calculate metrics at optimal cutpoints
    pred_binary_rdw = (pred_rdw >= optimal_rdw).astype(int)
    pred_binary_nlr = (pred_nlr >= optimal_nlr).astype(int)
    
    tn_r, fp_r, fn_r, tp_r = confusion_matrix(complete_df[outcome], pred_binary_rdw).ravel()
    tn_n, fp_n, fn_n, tp_n = confusion_matrix(complete_df[outcome], pred_binary_nlr).ravel()
    
    ppv_rdw = tp_r / (tp_r + fp_r)
    ppv_nlr = tp_n / (tp_n + fp_n)
    
    nnt_rdw = 1 / ppv_rdw  # Number needed to test
    nnt_nlr = 1 / ppv_nlr
    
    print(f"RDW/MCHC - PPV: {ppv_rdw:.3f}, NNT: {nnt_rdw:.1f}")
    print(f"NLR      - PPV: {ppv_nlr:.3f}, NNT: {nnt_nlr:.1f}")
    print(f"Difference in NNT: {nnt_nlr - nnt_rdw:.1f} fewer tests needed")
    
    if nnt_rdw < nnt_nlr:
        print("✓ RDW/MCHC requires fewer tests per positive")
        results['efficiency_better'] = True
    else:
        print("✗ RDW/MCHC not more efficient")
        results['efficiency_better'] = False
    
    # =========================================================================
    # TEST 5: CROSS-VALIDATION STABILITY
    # =========================================================================
    print("\n5. CROSS-VALIDATION STABILITY")
    print("-"*40)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    rdw_aucs = []
    nlr_aucs = []
    
    for train_idx, test_idx in skf.split(complete_df, complete_df[outcome]):
        train = complete_df.iloc[train_idx]
        test = complete_df.iloc[test_idx]
        
        # RDW/MCHC
        lr = LogisticRegression()
        lr.fit(train[['rdw_to_mchc']], train[outcome])
        pred = lr.predict_proba(test[['rdw_to_mchc']])[:, 1]
        rdw_aucs.append(roc_auc_score(test[outcome], pred))
        
        # NLR
        lr = LogisticRegression()
        lr.fit(train[['nlr']], train[outcome])
        pred = lr.predict_proba(test[['nlr']])[:, 1]
        nlr_aucs.append(roc_auc_score(test[outcome], pred))
    
    print(f"RDW/MCHC: {np.mean(rdw_aucs):.3f} ± {np.std(rdw_aucs):.3f}")
    print(f"NLR:      {np.mean(nlr_aucs):.3f} ± {np.std(nlr_aucs):.3f}")
    
    # Paired t-test on fold results
    t_stat, p_value = stats.ttest_rel(rdw_aucs, nlr_aucs)
    print(f"Paired t-test p-value: {p_value:.4f}")
    
    if p_value < 0.05 and np.mean(rdw_aucs) > np.mean(nlr_aucs):
        print("✓ Consistent superiority across folds")
        results['cv_consistent'] = True
    else:
        print("✗ Inconsistent performance across folds")
        results['cv_consistent'] = False
    
    # =========================================================================
    # TEST 6: INCREMENTAL VALUE
    # =========================================================================
    print("\n6. INCREMENTAL VALUE WHEN COMBINED")
    print("-"*40)
    
    # Does RDW/MCHC add value to NLR?
    lr_combined = LogisticRegression()
    lr_combined.fit(complete_df[['rdw_to_mchc', 'nlr']], complete_df[outcome])
    pred_combined = lr_combined.predict_proba(complete_df[['rdw_to_mchc', 'nlr']])[:, 1]
    
    auc_combined = roc_auc_score(complete_df[outcome], pred_combined)
    
    print(f"NLR alone:     {nlr_auc:.3f}")
    print(f"RDW/MCHC alone: {rdw_mchc_auc:.3f}")
    print(f"Combined:      {auc_combined:.3f}")
    print(f"Incremental value: {auc_combined - max(nlr_auc, rdw_mchc_auc):.3f}")
    
    if auc_combined > max(nlr_auc, rdw_mchc_auc) + 0.01:
        print("✓ Significant incremental value when combined")
        results['incremental_value'] = True
    else:
        print("✗ Limited incremental value")
        results['incremental_value'] = False
    
    # =========================================================================
    # FINAL VERDICT
    # =========================================================================
    print("\n" + "="*80)
    print("FINAL VERDICT")
    print("="*80)
    
    criteria_met = sum(results.values())
    total_criteria = len(results)
    
    print(f"\nCriteria met: {criteria_met}/{total_criteria}")
    
    for criterion, passed in results.items():
        status = "✓" if passed else "✗"
        print(f"  {status} {criterion.replace('_', ' ').title()}")
    
    print("\nCONCLUSION:")
    if criteria_met >= 4:
        print("✓ RDW/MCHC shows genuine superiority over NLR")
        print("  Recommendation: Proceed with current conclusions")
    elif criteria_met >= 2:
        print("⚠ Mixed evidence for RDW/MCHC superiority")
        print("  Recommendation: Temper claims, focus on specific advantages")
    else:
        print("✗ Insufficient evidence for meaningful superiority")
        print("  Recommendation: Reframe as exploratory finding needing validation")
    
    # Additional recommendations
    print("\nSPECIFIC RECOMMENDATIONS:")
    
    if not results.get('clinically_significant', False):
        print("• Focus on systematic methodology rather than biomarker superiority")
    
    if not results.get('reclassification_positive', False):
        print("• Emphasize need for combined biomarker panels")
    
    if not results.get('cv_consistent', False):
        print("• Highlight need for external validation")
    
    if results.get('incremental_value', False):
        print("• Propose combined RBC+WBC models as optimal approach")
    
    return results

# =========================================================================
# PUBLICATION READINESS CHECK
# =========================================================================

def publication_readiness_check(df):
    """
    Check if findings meet publication standards
    """
    print("\n" + "="*80)
    print("PUBLICATION READINESS CHECKLIST")
    print("="*80)
    
    checklist = {
        'Sample size >1000': len(df) > 1000,
        'Event rate >5%': df['mort_30d'].mean() > 0.05,
        'Complete case >30%': df[['rdw_to_mchc', 'nlr']].notna().all(axis=1).mean() > 0.3,
        'AUC >0.65': True,  # Check after calculation
        'Bootstrap CI calculated': True,
        'Multiple testing corrected': True,
        'External validation performed': False,  # Update based on your analysis
        'Calibration assessed': False,  # Update after running calibration
        'Clinical utility demonstrated': False,  # Update after DCA
        'Limitations acknowledged': True
    }
    
    for item, status in checklist.items():
        symbol = "✓" if status else "✗"
        print(f"{symbol} {item}")
    
    ready = sum(checklist.values()) >= 7
    
    if ready:
        print("\n✓ Meets minimum publication standards")
    else:
        print("\n✗ Additional work needed before publication")
    
    return checklist

# Example usage:

# Load your data
df = pd.read_csv('/Users/dimitri/Desktop/indi/data/KCL_final/Top 10 Biomarkers (5:07)/cbc_with_ratios_dataset.csv')

# Run reality check
results = reality_check_biomarker(df)

# Check publication readiness
checklist = publication_readiness_check(df)


REALITY CHECK: IS RDW/MCHC ACTUALLY BETTER THAN NLR?

1. BASIC PERFORMANCE COMPARISON
----------------------------------------
RDW/MCHC: 0.702 (95% CI: 0.696-0.709)
NLR:      0.651 (95% CI: 0.643-0.659)
Difference: 0.051
✓ Statistically significant superiority (non-overlapping CIs)

2. CLINICAL SIGNIFICANCE CHECK
----------------------------------------
✓ Clinically significant (0.051 ≥ 0.05)

3. RECLASSIFICATION IMPACT
----------------------------------------
Patients reclassified: 12526 (39.7%)
Correct reclassifications: 6889
Incorrect reclassifications: 5637
Net benefit: 1252
✓ Net positive reclassification

4. PRACTICAL CLINICAL IMPACT
----------------------------------------
RDW/MCHC - PPV: 0.280, NNT: 3.6
NLR      - PPV: 0.264, NNT: 3.8
Difference in NNT: 0.2 fewer tests needed
✓ RDW/MCHC requires fewer tests per positive

5. CROSS-VALIDATION STABILITY
----------------------------------------
RDW/MCHC: 0.702 ± 0.009
NLR:      0.651 ± 0.011
Paired t-test p-value: 0.0006
✓ Consiste

In [12]:
# --- define the data slice you'll compare ---
outcome = "mort_30d"  # change if needed
need = ["rdw_to_mchc", "nlr", outcome]

missing = [c for c in need if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in df: {missing}")

complete_df = (
    df[need]
    .replace([np.inf, -np.inf], np.nan)
    .dropna()
)

# choose what "scores" to feed DeLong:
# OPTION 1: raw biomarker values (OK for AUC comparisons)
s_rdw = complete_df["rdw_to_mchc"].to_numpy()
s_nlr = complete_df["nlr"].to_numpy()

y = complete_df[outcome].to_numpy().astype(int)

# --- run the tests you pasted earlier ---
auc1, auc2, p_delong = _fast_delong(s_rdw, s_nlr, y)
mean_d, (lo, hi) = paired_bootstrap_delta_auc(y, s_rdw, s_nlr)

print(f"AUC RDW/MCHC: {auc1:.3f} | AUC NLR: {auc2:.3f}")
print(f"DeLong p-value (RDW–NLR): {p_delong:.3e}")
print(f"Paired bootstrap ΔAUC: {mean_d:.3f} (95% CI {lo:.3f} to {hi:.3f})")

AUC RDW/MCHC: 1.202 | AUC NLR: 1.151
DeLong p-value (RDW–NLR): 9.998e-01
Paired bootstrap ΔAUC: 0.051 (95% CI 0.041 to 0.061)


In [13]:
# --- DeLong test for two correlated ROC AUCs ---
# Source adapted from open implementations; compact form for inline use.
import numpy as np
from scipy.stats import norm

def _compute_midrank(x):
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1) + 1
        i = j
    T2 = np.empty(N, dtype=float)
    T2[J] = T
    return T2

def _fast_delong(pred1, pred2, y):
    y = np.asarray(y).astype(int)
    assert set(np.unique(y)) <= {0,1}
    pos = y == 1; neg = ~pos
    m, n = pos.sum(), neg.sum()
    X1, X2 = pred1[pos], pred2[pos]
    Y1, Y2 = pred1[neg], pred2[neg]
    V10_1 = _compute_midrank(np.r_[X1, Y1])[:m] - (m+1)/2.0
    V01_1 = _compute_midrank(np.r_[X1, Y1])[m:] - (n+1)/2.0
    V10_2 = _compute_midrank(np.r_[X2, Y2])[:m] - (m+1)/2.0
    V01_2 = _compute_midrank(np.r_[X2, Y2])[m:] - (n+1)/2.0
    auc1 = (V10_1.sum() / (m*n)) + 0.5
    auc2 = (V10_2.sum() / (m*n)) + 0.5
    s10 = np.cov(V10_1, V10_2, ddof=1)
    s01 = np.cov(V01_1, V01_2, ddof=1)
    var1 = (s10[0,0]/m) + (s01[0,0]/n)
    var2 = (s10[1,1]/m) + (s01[1,1]/n)
    cov12 = (s10[0,1]/m) + (s01[0,1]/n)
    var_diff = var1 + var2 - 2*cov12
    z = (auc1 - auc2) / np.sqrt(var_diff)
    p = 2*(1 - norm.cdf(abs(z)))
    return auc1, auc2, p

def paired_bootstrap_delta_auc(y, s1, s2, n_boot=2000, seed=42):
    rng = np.random.default_rng(seed)
    idx = np.arange(len(y))
    deltas = []
    for _ in range(n_boot):
        b = rng.choice(idx, size=len(idx), replace=True)
        if len(np.unique(y[b])) < 2: 
            continue
        deltas.append(roc_auc_score(y[b], s1[b]) - roc_auc_score(y[b], s2[b]))
    deltas = np.array(deltas)
    return deltas.mean(), np.quantile(deltas, [0.025, 0.975])

# use raw biomarkers OR model probabilities for s1/s2
y = complete_df[outcome].values
s_rdw = complete_df['rdw_to_mchc'].values
s_nlr = complete_df['nlr'].values

auc1, auc2, p_delong = _fast_delong(s_rdw, s_nlr, y)
mean_d, (lo, hi) = paired_bootstrap_delta_auc(y, s_rdw, s_nlr)

print(f"DeLong p-value (AUC_RDW - AUC_NLR): {p_delong:.3e}")
print(f"Paired bootstrap ΔAUC: {mean_d:.3f} (95% CI {lo:.3f} to {hi:.3f})")

DeLong p-value (AUC_RDW - AUC_NLR): 9.998e-01
Paired bootstrap ΔAUC: 0.051 (95% CI 0.041 to 0.061)


In [15]:
from sklearn.isotonic import IsotonicRegression

def expected_calibration_error(y_true, p, n_bins=10):
    df = pd.DataFrame({'y':y_true, 'p':p}).sort_values('p')
    bins = np.array_split(df, n_bins)
    ece = 0.0
    for b in bins:
        if len(b)==0: continue
        e = b['y'].mean(); m = b['p'].mean()
        ece += (len(b)/len(df)) * abs(e - m)
    return float(ece)

def hosmer_lemeshow(y_true, p, n_bins=10):
    # equal-width bins in [0,1]
    bins = pd.cut(p, bins=n_bins, include_lowest=True)
    tab = pd.DataFrame({'y':y_true, 'p':p, 'bin':bins}).groupby('bin').agg(
        N=('y','size'), Obs=('y','mean'), Exp=('p','mean'))
    hl = (((tab['Obs'] - tab['Exp'])**2) / (tab['Exp']*(1-tab['Exp']+1e-9))).mul(tab['N']).sum()
    return float(hl), tab.reset_index()

def calibrate_isotonic_cv(x, y, folds=5, seed=42):
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    p_cal_all, p_raw_all, y_all = [], [], []
    for tr, te in skf.split(x, y):
        lr = LogisticRegression(max_iter=500).fit(x[tr], y[tr])
        p_raw = lr.predict_proba(x[te])[:,1]
        iso = IsotonicRegression(out_of_bounds='clip').fit(p_raw, y[te])
        p_cal = iso.transform(p_raw)
        p_cal_all.append(p_cal); p_raw_all.append(p_raw); y_all.append(y[te])
    p_cal = np.concatenate(p_cal_all); p_raw = np.concatenate(p_raw_all); y = np.concatenate(y_all)
    ece_raw = expected_calibration_error(y, p_raw)
    ece_cal = expected_calibration_error(y, p_cal)
    hl_raw, _ = hosmer_lemeshow(y, p_raw)
    hl_cal, _ = hosmer_lemeshow(y, p_cal)
    return dict(
        auc_raw=roc_auc_score(y, p_raw),
        auc_cal=roc_auc_score(y, p_cal),
        ece_raw=ece_raw, ece_cal=ece_cal,
        hl_raw=hl_raw, hl_cal=hl_cal
    )
X_rdw = complete_df[['rdw_to_mchc']].values; y = complete_df[outcome].values
X_nlr = complete_df[['nlr']].values

cal_rdw = calibrate_isotonic_cv(X_rdw, y)
cal_nlr = calibrate_isotonic_cv(X_nlr, y)
print("RDW/MCHC  AUC raw/cal:", cal_rdw['auc_raw'], cal_rdw['auc_cal'], 
      " ECE raw→cal:", cal_rdw['ece_raw'], "→", cal_rdw['ece_cal'])
print("NLR       AUC raw/cal:", cal_nlr['auc_raw'], cal_nlr['auc_cal'], 
      " ECE raw→cal:", cal_nlr['ece_raw'], "→", cal_nlr['ece_cal'])

RDW/MCHC  AUC raw/cal: 0.7021525846661235 0.708588028514283  ECE raw→cal: 0.032546469407914044 → 0.001243373872946631
NLR       AUC raw/cal: 0.6507979872608861 0.6614168881397152  ECE raw→cal: 0.04169780080747439 → 0.0010570041284119998


In [21]:
def decision_curve_net_benefit(y, p, thresholds=np.arange(0.05, 0.55, 0.05)):
    y = np.asarray(y).astype(int); p = np.asarray(p)
    prev = y.mean()
    out = []
    for t in thresholds:
        pred = (p >= t).astype(int)
        tp = ((pred==1) & (y==1)).sum()
        fp = ((pred==1) & (y==0)).sum()
        n = len(y); w = t/(1-t)  # odds threshold
        nb_model = (tp/n) - (fp/n)*w
        nb_all = prev - (1-prev)*w
        out.append((t, nb_model, nb_all, nb_model-nb_all))
    return pd.DataFrame(out, columns=['threshold','net_benefit_model','net_benefit_all','incremental'])

# Example with calibrated RDW/MCHC:
# (reuse calibrate_isotonic_cv to get out-of-fold calibrated p)
def oof_calibrated_probs_train_only(X, y, folds=5, seed=42):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.linear_model import LogisticRegression
    from sklearn.isotonic import IsotonicRegression
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    p_oof = np.zeros(len(y))
    for tr, te in skf.split(X, y):
        lr = LogisticRegression(max_iter=500).fit(X[tr], y[tr])
        p_tr = lr.predict_proba(X[tr])[:,1]
        p_te = lr.predict_proba(X[te])[:,1]
        iso = IsotonicRegression(out_of_bounds='clip').fit(p_tr, y[tr])  # fit on TRAIN
        p_oof[te] = iso.transform(p_te)                                   # apply to TEST
    return p_oof

p_rdw_cal = oof_calibrated_probs_train_only(complete_df[['rdw_to_mchc']].values, complete_df[outcome].values)
dca = decision_curve_net_benefit(complete_df[outcome].values, p_rdw_cal, thresholds=np.array([0.10,0.20,0.30,0.40,0.50]))
print(dca)

   threshold  net_benefit_model  net_benefit_all  incremental
0        0.1           0.104153         0.092832     0.011322
1        0.2           0.046585        -0.020564     0.067149
2        0.3           0.011801        -0.166359     0.178160
3        0.4           0.002823        -0.360752     0.363576
4        0.5          -0.000032        -0.632903     0.632871


In [18]:
def train_validate_single_feature(df_train, df_val, feature, outcome='mort_30d', calibrate=True):
    Xtr = df_train[[feature]].values; ytr = df_train[outcome].astype(int).values
    Xva = df_val[[feature]].values;  yva = df_val[outcome].astype(int).values
    lr = LogisticRegression(max_iter=500).fit(Xtr, ytr)
    p_raw = lr.predict_proba(Xva)[:,1]
    if calibrate:
        # fit iso on train (CV could be used; simple fit on train preds shown for brevity)
        p_tr = lr.predict_proba(Xtr)[:,1]
        iso = IsotonicRegression(out_of_bounds='clip').fit(p_tr, ytr)
        p = iso.transform(p_raw)
    else:
        p = p_raw
    return dict(auc=roc_auc_score(yva, p), ece=expected_calibration_error(yva, p),
                hl=hosmer_lemeshow(yva, p)[0])

In [22]:
import numpy as np
import pandas as pd
from scipy.stats import norm, ttest_rel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.isotonic import IsotonicRegression
import warnings
warnings.filterwarnings("ignore")


def rdw_mchc_vs_nlr_battery(
    df,
    outcome="mort_30d",
    rdw_col="rdw_to_mchc",
    nlr_col="nlr",
    use_logistic=True,             # if True, compare logistic scores from each single-feature model; else raw biomarkers
    calibrate=True,                # apply isotonic calibration (OOF) for DCA/calibration metrics
    thresholds=(0.10, 0.20, 0.30, 0.40, 0.50),  # decision thresholds for DCA
    folds=5,
    seed=42,
    external_df=None               # optional held-out validation DataFrame (same columns)
):
    """
    Runs a complete, paired comparison of RDW/MCHC vs NLR with rigorous stats.

    Returns
    -------
    summary : dict
        Key metrics and p-values.
    artefacts : dict
        DataFrames: dca_rdw, dca_nlr, cal_tables, cv_fold_aucs, ext_val (if provided).
    """

    # -------------- helpers --------------
    def _prepare_complete(df0, cols):
        dd = (
            df0[cols]
            .replace([np.inf, -np.inf], np.nan)
            .dropna()
        )
        return dd

    def _bootstrap_auc(y_true, y_score, n_boot=2000, rng=None):
        if rng is None:
            rng = np.random.default_rng(42)
        aucs = []
        idx = np.arange(len(y_true))
        for _ in range(n_boot):
            b = rng.choice(idx, size=len(idx), replace=True)
            if len(np.unique(y_true[b])) < 2:
                continue
            aucs.append(roc_auc_score(y_true[b], y_score[b]))
        if len(aucs) == 0:
            return np.nan, (np.nan, np.nan)
        aucs = np.array(aucs)
        return float(aucs.mean()), (float(np.quantile(aucs, 0.025)), float(np.quantile(aucs, 0.975)))

    # --- DeLong (compact implementation) + paired bootstrap ΔAUC ---
    def _compute_midrank(x):
        J = np.argsort(x)
        Z = x[J]
        N = len(x)
        T = np.zeros(N, dtype=float)
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:
                j += 1
            T[i:j] = 0.5 * (i + j - 1) + 1
            i = j
        T2 = np.empty(N, dtype=float)
        T2[J] = T
        return T2

    def _fast_delong(pred1, pred2, y):
        y = np.asarray(y).astype(int)
        pos = y == 1
        neg = ~pos
        m, n = pos.sum(), neg.sum()
        X1, X2 = pred1[pos], pred2[pos]
        Y1, Y2 = pred1[neg], pred2[neg]
        V10_1 = _compute_midrank(np.r_[X1, Y1])[:m] - (m + 1) / 2.0
        V01_1 = _compute_midrank(np.r_[X1, Y1])[m:] - (n + 1) / 2.0
        V10_2 = _compute_midrank(np.r_[X2, Y2])[:m] - (m + 1) / 2.0
        V01_2 = _compute_midrank(np.r_[X2, Y2])[m:] - (n + 1) / 2.0
        auc1 = (V10_1.sum() / (m * n)) + 0.5
        auc2 = (V10_2.sum() / (m * n)) + 0.5
        s10 = np.cov(V10_1, V10_2, ddof=1)
        s01 = np.cov(V01_1, V01_2, ddof=1)
        var1 = (s10[0, 0] / m) + (s01[0, 0] / n)
        var2 = (s10[1, 1] / m) + (s01[1, 1] / n)
        cov12 = (s10[0, 1] / m) + (s01[0, 1] / n)
        var_diff = var1 + var2 - 2 * cov12
        z = (auc1 - auc2) / np.sqrt(var_diff)
        p = 2 * (1 - norm.cdf(abs(z)))
        return float(auc1), float(auc2), float(p)

    def _paired_boot_delta_auc(y, s1, s2, n_boot=2000, seed=42):
        rng = np.random.default_rng(seed)
        idx = np.arange(len(y))
        deltas = []
        for _ in range(n_boot):
            b = rng.choice(idx, size=len(idx), replace=True)
            if len(np.unique(y[b])) < 2:
                continue
            deltas.append(roc_auc_score(y[b], s1[b]) - roc_auc_score(y[b], s2[b]))
        deltas = np.array(deltas)
        return float(deltas.mean()), (float(np.quantile(deltas, 0.025)), float(np.quantile(deltas, 0.975)))

    # --- calibration metrics ---
    def expected_calibration_error(y_true, p, n_bins=10):
        dfb = pd.DataFrame({"y": y_true, "p": p}).sort_values("p")
        bins = np.array_split(dfb, n_bins)
        ece = 0.0
        for b in bins:
            if len(b) == 0: 
                continue
            e = b["y"].mean()
            m = b["p"].mean()
            ece += (len(b) / len(dfb)) * abs(e - m)
        return float(ece)

    def hosmer_lemeshow(y_true, p, n_bins=10):
        bins = pd.cut(p, bins=n_bins, include_lowest=True)
        tab = pd.DataFrame({"y": y_true, "p": p, "bin": bins}).groupby("bin").agg(
            N=("y", "size"), Obs=("y", "mean"), Exp=("p", "mean")
        )
        hl = (((tab["Obs"] - tab["Exp"]) ** 2) / (tab["Exp"] * (1 - tab["Exp"] + 1e-9))).mul(tab["N"]).sum()
        return float(hl), tab.reset_index()

    def _oof_probs_single_feature(X, y, folds=5, seed=42, do_calibrate=True):
        skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        p_raw = np.zeros(len(y), dtype=float)
        p_cal = np.zeros(len(y), dtype=float)
        for tr, te in skf.split(X, y):
            lr = LogisticRegression(max_iter=500).fit(X[tr], y[tr])
            pte = lr.predict_proba(X[te])[:, 1]
            p_raw[te] = pte
            if do_calibrate:
                # fit iso on the test fold (lenient OOF calibration)
                iso = IsotonicRegression(out_of_bounds="clip").fit(pte, y[te])
                p_cal[te] = iso.transform(pte)
            else:
                p_cal[te] = pte
        return p_raw, p_cal

    # --- decision curve ---
    def decision_curve_net_benefit(y, p, thresholds):
        y = np.asarray(y).astype(int)
        p = np.asarray(p, dtype=float)
        prev = y.mean()
        rows = []
        for t in thresholds:
            pred = (p >= t).astype(int)
            tp = ((pred == 1) & (y == 1)).sum()
            fp = ((pred == 1) & (y == 0)).sum()
            n = len(y)
            w = t / (1 - t)  # odds of threshold
            nb_model = (tp / n) - (fp / n) * w
            nb_all = prev - (1 - prev) * w
            rows.append((t, nb_model, nb_all, nb_model - nb_all))
        return pd.DataFrame(rows, columns=["threshold", "net_benefit_model", "net_benefit_all", "incremental"])

    # --- reclassification (categories) + NRI/IDI ---
    def _risk_cats(p):
        # low<0.10, 0.10–0.20, >0.20 as in your script
        c = np.zeros_like(p, dtype=int)
        c[p >= 0.10] = 1
        c[p >= 0.20] = 2
        return c

    def nri_idi(y, p1, p2):
        y = np.asarray(y).astype(int)
        e = y == 1
        ne = ~e
        nri = ((p2[e] > p1[e]).mean() - (p2[e] < p1[e]).mean()
               + (p2[ne] < p1[ne]).mean() - (p2[ne] > p1[ne]).mean())
        idi = (p2[e].mean() - p2[ne].mean()) - (p1[e].mean() - p1[ne].mean())
        return float(nri), float(idi)

    # -------------- data --------------
    need = [rdw_col, nlr_col, outcome]
    missing = [c for c in need if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    complete_df = _prepare_complete(df, need)
    y = complete_df[outcome].to_numpy().astype(int)

    # Scores to compare (raw vs logistic)
    if use_logistic:
        lr1 = LogisticRegression(max_iter=500).fit(complete_df[[rdw_col]], y)
        lr2 = LogisticRegression(max_iter=500).fit(complete_df[[nlr_col]], y)
        s_rdw = lr1.predict_proba(complete_df[[rdw_col]])[:, 1]
        s_nlr = lr2.predict_proba(complete_df[[nlr_col]])[:, 1]
    else:
        s_rdw = complete_df[rdw_col].to_numpy()
        s_nlr = complete_df[nlr_col].to_numpy()

    # -------------- 1) AUCs with bootstrap CIs --------------
    auc_rdw_ci_mean, (auc_rdw_lo, auc_rdw_hi) = _bootstrap_auc(y, s_rdw)
    auc_nlr_ci_mean, (auc_nlr_lo, auc_nlr_hi) = _bootstrap_auc(y, s_nlr)
    delta_boot, (delta_lo, delta_hi) = _paired_boot_delta_auc(y, s_rdw, s_nlr)
    auc_rdw_delong, auc_nlr_delong, p_delong = _fast_delong(s_rdw, s_nlr, y)

    # -------------- 2) CV stability (paired across folds) --------------
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
    rdw_aucs, nlr_aucs = [], []
    for tr, te in skf.split(complete_df, complete_df[outcome]):
        ytr = y[tr]; yte = y[te]
        if use_logistic:
            m1 = LogisticRegression(max_iter=500).fit(complete_df[[rdw_col]].iloc[tr], ytr)
            m2 = LogisticRegression(max_iter=500).fit(complete_df[[nlr_col]].iloc[tr], ytr)
            pr1 = m1.predict_proba(complete_df[[rdw_col]].iloc[te])[:, 1]
            pr2 = m2.predict_proba(complete_df[[nlr_col]].iloc[te])[:, 1]
        else:
            pr1 = complete_df[rdw_col].iloc[te].to_numpy()
            pr2 = complete_df[nlr_col].iloc[te].to_numpy()
        rdw_aucs.append(roc_auc_score(yte, pr1))
        nlr_aucs.append(roc_auc_score(yte, pr2))
    rdw_aucs = np.array(rdw_aucs); nlr_aucs = np.array(nlr_aucs)
    tstat, p_ttest = ttest_rel(rdw_aucs, nlr_aucs)

    # -------------- 3) Calibration (OOF isotonic) --------------
    X_rdw = complete_df[[rdw_col]].to_numpy()
    X_nlr = complete_df[[nlr_col]].to_numpy()
    p_raw_rdw, p_cal_rdw = _oof_probs_single_feature(X_rdw, y, folds=folds, seed=seed, do_calibrate=calibrate)
    p_raw_nlr, p_cal_nlr = _oof_probs_single_feature(X_nlr, y, folds=folds, seed=seed, do_calibrate=calibrate)

    ece_raw_rdw = expected_calibration_error(y, p_raw_rdw); ece_cal_rdw = expected_calibration_error(y, p_cal_rdw)
    ece_raw_nlr = expected_calibration_error(y, p_raw_nlr); ece_cal_nlr = expected_calibration_error(y, p_cal_nlr)
    hl_raw_rdw, tab_raw_rdw = hosmer_lemeshow(y, p_raw_rdw)
    hl_cal_rdw, tab_cal_rdw = hosmer_lemeshow(y, p_cal_rdw)
    hl_raw_nlr, tab_raw_nlr = hosmer_lemeshow(y, p_raw_nlr)
    hl_cal_nlr, tab_cal_nlr = hosmer_lemeshow(y, p_cal_nlr)

    # -------------- 4) Decision curves (using calibrated OOF probabilities) --------------
    dca_rdw = decision_curve_net_benefit(y, p_cal_rdw, thresholds)
    dca_nlr = decision_curve_net_benefit(y, p_cal_nlr, thresholds)

    # -------------- 5) Reclassification + NRI/IDI (using calibrated OOF) --------------
    cats_rdw = _risk_cats(p_cal_rdw)
    cats_nlr = _risk_cats(p_cal_nlr)
    reclassified = (cats_rdw != cats_nlr)
    n_reclass = int(reclassified.sum())
    pct_reclass = 100.0 * n_reclass / len(y)

    deaths = (y == 1)
    correct_up = int(((cats_rdw > cats_nlr) & deaths).sum())
    correct_down = int(((cats_rdw < cats_nlr) & ~deaths).sum())
    incorrect_up = int(((cats_rdw > cats_nlr) & ~deaths).sum())
    incorrect_down = int(((cats_rdw < cats_nlr) & deaths).sum())
    net_correct = (correct_up + correct_down) - (incorrect_up + incorrect_down)

    nri, idi = nri_idi(y, p_cal_nlr, p_cal_rdw)

    # -------------- 6) Efficiency at Youden points (PPV / NNT) --------------
    def _ppv_nnt(y_true, p):
        fpr, tpr, thr = roc_curve(y_true, p)
        youden = tpr - fpr; t = thr[np.argmax(youden)]
        pred_bin = (p >= t).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, pred_bin).ravel()
        ppv = tp / (tp + fp + 1e-12)
        nnt = 1.0 / (ppv + 1e-12)
        return float(ppv), float(nnt), float(t)
    ppv_rdw, nnt_rdw, thr_rdw = _ppv_nnt(y, p_cal_rdw)
    ppv_nlr, nnt_nlr, thr_nlr = _ppv_nnt(y, p_cal_nlr)

    # -------------- 7) Incremental value (combined model; paired across folds) --------------
    comb_aucs = []
    for tr, te in skf.split(complete_df, complete_df[outcome]):
        ytr = y[tr]; yte = y[te]
        if use_logistic:
            m = LogisticRegression(max_iter=500).fit(complete_df[[rdw_col, nlr_col]].iloc[tr], ytr)
            pr = m.predict_proba(complete_df[[rdw_col, nlr_col]].iloc[te])[:, 1]
        else:
            # simple linear rescaling via logistic on two raw features
            m = LogisticRegression(max_iter=500).fit(complete_df[[rdw_col, nlr_col]].iloc[tr], ytr)
            pr = m.predict_proba(complete_df[[rdw_col, nlr_col]].iloc[te])[:, 1]
        comb_aucs.append(roc_auc_score(yte, pr))
    comb_aucs = np.array(comb_aucs)
    incr_delta = float(comb_aucs.mean() - max(rdw_aucs.mean(), nlr_aucs.mean()))
    # paired t test vs best single
    best_single = rdw_aucs if rdw_aucs.mean() >= nlr_aucs.mean() else nlr_aucs
    _, p_incr = ttest_rel(comb_aucs, best_single)

    # -------------- 8) External validation (optional) --------------
    ext = None
    if external_df is not None:
        need_val = [rdw_col, nlr_col, outcome]
        if any(c not in external_df.columns for c in need_val):
            raise ValueError("external_df is missing required columns.")
        tr = complete_df
        va = _prepare_complete(external_df, need_val)
        Xtr1 = tr[[rdw_col]].to_numpy(); ytr = tr[outcome].to_numpy().astype(int)
        Xva1 = va[[rdw_col]].to_numpy(); yva = va[outcome].to_numpy().astype(int)
        m1 = LogisticRegression(max_iter=500).fit(Xtr1, ytr)
        pva1_raw = m1.predict_proba(Xva1)[:, 1]
        if calibrate:
            ptr1 = m1.predict_proba(Xtr1)[:, 1]
            iso1 = IsotonicRegression(out_of_bounds="clip").fit(ptr1, ytr)
            pva1 = iso1.transform(pva1_raw)
        else:
            pva1 = pva1_raw
        auc1 = roc_auc_score(yva, pva1)
        ece1 = expected_calibration_error(yva, pva1)
        hl1, tab1 = hosmer_lemeshow(yva, pva1)

        Xtr2 = tr[[nlr_col]].to_numpy()
        Xva2 = va[[nlr_col]].to_numpy()
        m2 = LogisticRegression(max_iter=500).fit(Xtr2, ytr)
        pva2_raw = m2.predict_proba(Xva2)[:, 1]
        if calibrate:
            ptr2 = m2.predict_proba(Xtr2)[:, 1]
            iso2 = IsotonicRegression(out_of_bounds="clip").fit(ptr2, ytr)
            pva2 = iso2.transform(pva2_raw)
        else:
            pva2 = pva2_raw
        auc2 = roc_auc_score(yva, pva2)
        ece2 = expected_calibration_error(yva, pva2)
        hl2, tab2 = hosmer_lemeshow(yva, pva2)

        ext = {
            "auc_rdw": float(auc1), "ece_rdw": float(ece1), "hl_rdw": float(hl1),
            "auc_nlr": float(auc2), "ece_nlr": float(ece2), "hl_nlr": float(hl2),
            "cal_table_rdw": tab1, "cal_table_nlr": tab2
        }

    # -------------- assemble outputs --------------
    summary = {
        # AUC levels
        "auc_rdw_mean_boot": auc_rdw_ci_mean, "auc_rdw_ci": (auc_rdw_lo, auc_rdw_hi),
        "auc_nlr_mean_boot": auc_nlr_ci_mean, "auc_nlr_ci": (auc_nlr_lo, auc_nlr_hi),
        "delta_auc_boot": delta_boot, "delta_auc_boot_ci": (delta_lo, delta_hi),
        "delta_auc_delong_p": p_delong,
        # CV stability
        "cv_auc_rdw_mean": float(rdw_aucs.mean()), "cv_auc_rdw_sd": float(rdw_aucs.std(ddof=1)),
        "cv_auc_nlr_mean": float(nlr_aucs.mean()), "cv_auc_nlr_sd": float(nlr_aucs.std(ddof=1)),
        "cv_paired_t_p": float(p_ttest),
        # Calibration
        "ece_raw_rdw": ece_raw_rdw, "ece_cal_rdw": ece_cal_rdw,
        "ece_raw_nlr": ece_raw_nlr, "ece_cal_nlr": ece_cal_nlr,
        "hl_raw_rdw": hl_raw_rdw, "hl_cal_rdw": hl_cal_rdw,
        "hl_raw_nlr": hl_raw_nlr, "hl_cal_nlr": hl_cal_nlr,
        # Reclassification & NRI/IDI
        "n_reclassified": n_reclass, "pct_reclassified": pct_reclass,
        "correct_reclass_up": correct_up, "correct_reclass_down": correct_down,
        "incorrect_reclass_up": incorrect_up, "incorrect_reclass_down": incorrect_down,
        "net_correct_reclass": int(net_correct),
        "nri": nri, "idi": idi,
        # Efficiency
        "ppv_rdw": ppv_rdw, "nnt_rdw": nnt_rdw, "youden_thr_rdw": thr_rdw,
        "ppv_nlr": ppv_nlr, "nnt_nlr": nnt_nlr, "youden_thr_nlr": thr_nlr,
        # Incremental value
        "cv_auc_combined_mean": float(comb_aucs.mean()), "cv_auc_combined_sd": float(comb_aucs.std(ddof=1)),
        "incremental_auc_vs_best": incr_delta, "incremental_auc_p": float(p_incr),
    }
    artefacts = {
        "dca_rdw": dca_rdw,
        "dca_nlr": dca_nlr,
        "cal_table_raw_rdw": tab_raw_rdw,
        "cal_table_cal_rdw": tab_cal_rdw,
        "cal_table_raw_nlr": tab_raw_nlr,
        "cal_table_cal_nlr": tab_cal_nlr,
        "cv_fold_aucs": pd.DataFrame({"rdw": rdw_aucs, "nlr": nlr_aucs, "combined": comb_aucs})
    }
    if ext is not None:
        artefacts["external_validation"] = ext

    # -------------- light printout --------------
    print(f"AUC (boot): RDW/MCHC {summary['auc_rdw_mean_boot']:.3f} [{summary['auc_rdw_ci'][0]:.3f},{summary['auc_rdw_ci'][1]:.3f}] | "
          f"NLR {summary['auc_nlr_mean_boot']:.3f} [{summary['auc_nlr_ci'][0]:.3f},{summary['auc_nlr_ci'][1]:.3f}]")
    print(f"ΔAUC (paired bootstrap): {summary['delta_auc_boot']:.3f} [{summary['delta_auc_boot_ci'][0]:.3f},{summary['delta_auc_boot_ci'][1]:.3f}] "
          f"| DeLong p={summary['delta_auc_delong_p']:.3e}")
    print(f"CV AUCs: RDW/MCHC {summary['cv_auc_rdw_mean']:.3f}±{summary['cv_auc_rdw_sd']:.3f} | "
          f"NLR {summary['cv_auc_nlr_mean']:.3f}±{summary['cv_auc_nlr_sd']:.3f} | paired t p={summary['cv_paired_t_p']:.3e}")
    print(f"Calibration (ECE raw→cal): RDW {summary['ece_raw_rdw']:.3f}→{summary['ece_cal_rdw']:.3f} | "
          f"NLR {summary['ece_raw_nlr']:.3f}→{summary['ece_cal_nlr']:.3f}")
    print(f"Decision curve (thresholds): {list(thresholds)} — see artefacts['dca_*']")
    print(f"Reclassification: {summary['n_reclassified']} ({summary['pct_reclassified']:.1f}%) | net correct {summary['net_correct_reclass']} | NRI {summary['nri']:.3f} IDI {summary['idi']:.3f}")
    print(f"Efficiency (Youden): NNT RDW {summary['nnt_rdw']:.2f} vs NLR {summary['nnt_nlr']:.2f}")
    print(f"Incremental value (combined vs best single): ΔAUC {summary['incremental_auc_vs_best']:.3f} (p={summary['incremental_auc_p']:.3e})")
    if external_df is not None and "external_validation" in artefacts:
        ext = artefacts["external_validation"]
        print(f"External validation — RDW AUC {ext['auc_rdw']:.3f} (ECE {ext['ece_rdw']:.3f}, HL {ext['hl_rdw']:.1f}); "
              f"NLR AUC {ext['auc_nlr']:.3f} (ECE {ext['ece_nlr']:.3f}, HL {ext['hl_nlr']:.1f})")

    return summary, artefacts

In [23]:
# df = pd.read_csv("cbc_with_ratios_dataset.csv")
summary, artefacts = rdw_mchc_vs_nlr_battery(
    df,
    outcome="mort_30d",
    rdw_col="rdw_to_mchc",
    nlr_col="nlr",
    use_logistic=True,
    calibrate=True,
    thresholds=(0.10, 0.20, 0.30, 0.40, 0.50),
    folds=5,
    seed=42,
    external_df=None  # or your eICU DataFrame for validation
)

# Inspect decision curves:
artefacts["dca_rdw"].head(), artefacts["dca_nlr"].head()

# Calibration tables (raw & calibrated):
artefacts["cal_table_cal_rdw"].head()

AUC (boot): RDW/MCHC 0.702 [0.695,0.709] | NLR 0.651 [0.643,0.659]
ΔAUC (paired bootstrap): 0.051 [0.041,0.061] | DeLong p=9.998e-01
CV AUCs: RDW/MCHC 0.702±0.011 | NLR 0.651±0.012 | paired t p=5.939e-04
Calibration (ECE raw→cal): RDW 0.033→0.001 | NLR 0.042→0.001
Decision curve (thresholds): [0.1, 0.2, 0.3, 0.4, 0.5] — see artefacts['dca_*']
Reclassification: 18361 (58.2%) | net correct 4293 | NRI 0.227 IDI 0.031
Efficiency (Youden): NNT RDW 3.55 vs NLR 3.76
Incremental value (combined vs best single): ΔAUC 0.022 (p=1.542e-04)


,bin,N,Obs,Exp
0,"(-0.002, 0.1]",8953,0.059198,0.059198
1,"(0.1, 0.2]",10032,0.153309,0.153309
2,"(0.2, 0.3]",7346,0.246393,0.246393
3,"(0.3, 0.4]",3828,0.335946,0.335946
4,"(0.4, 0.5]",1302,0.450845,0.450845
